# Transfering Data to Snowflake

In [1]:
import os
import json
from snowflake.snowpark.session import Session

params = json.loads(os.environ.get("SNOWFLAKE_CONNECTION"))
session = Session.builder.configs(params).create()

In [ ]:
import json
from pathlib import Path

GEOJSON_PATH = "path/to/your_file.geojson"   # <- edit this
TABLE = "XD_GEOM"                           # existing table
